# Week 8: Spatial interpolation and reading data from online

In [1]:
# spatial interpolation
import numpy as np
import scipy.interpolate as sciint
import scipy.special as sps
import pylab
import matplotlib.pyplot as plt

# import data from online
import urllib, json
from datetime import datetime

%matplotlib inline

## Theis equation

Last time, we created a spatial map of two wells pumping, using the Theis Equation, see below.

In [2]:
Q = 133000 # cfd
T = 1200 #ft^2/d
Ss = 2.6*10**-7 #1/ft
b = 200 # ft
S = Ss*b 
t = 1 #day

# function that superimposes two Theis Equations
def theis(x,y,Q,T,S,t):
    return Q/(4*np.pi*T)*sps.exp1(((x**2+y**2))*S/(4*T*t))+Q/(4*np.pi*T)*sps.exp1((((x-2500)**2+(y+2500)**2))*S/(4*T*t))

# create numpy arrays so we can plot the Theis Equation over a 100 ft grid from -5050 ft to 5050 ft, both in the x and y directions
xpoints = np.arange(-5050,5051,100)
ypoints = np.arange(-5050,5051,100)
# create a mesh grid based on the arrays defined above
X,Y = pylab.meshgrid(xpoints,ypoints)
# create head values using the Theis function for every point on the grid
values = theis(X,Y,Q,T,S,t)

# control size of image
pylab.rcParams['figure.figsize'] = 10,10
# show the head values in a 2D image
# we define the color map (cmap) with a yellow, orange, red color bar
# color bar options are discussed at: https://matplotlib.org/examples/color/colormaps_reference.html
im = pylab.imshow(values,cmap=pylab.cm.YlOrRd,origin='lower')
# contour the data in intervals of 5 from 0 to 200
cset = pylab.contour(values,np.arange(0,201,5))
# display the color bar
pylab.colorbar(im)
# add a title to our image
pylab.title('Theis solution')
# show the image
pylab.show()

## Create a random distribution of points over the same spatial domain 

In [3]:
# np.random.rand(numberofpoints,dimensions)
# this function creates an array of points from 0 to 1
# to plot from -5050 to 5050, we must multiple the randomly generated points by 10,100 and then subtract 5,050
rpoints = 10100*np.random.rand(500, 2)-5050

## Plot the randomly generated points on a standard x,y plot 

In [4]:
# define the size of the plot
plt.rcParams['figure.figsize'] = 8,8
# plot the location of each point as a black dot
plt.plot(rpoints[:,0], rpoints[:,1], 'k.')
plt.show()

## Calculate the Theis Solution at each randomly generated point

In [5]:
# Calculate Theis for randomly generated points using the function that we already generated
ptvalues = theis(rpoints[:,0], rpoints[:,1], Q, T, S, t)
# interpolate the data over a grid defined by X and Y
# griddata(x and y inputs, values, domain over which to plot data, interpolation method)
grid_z0 = sciint.griddata(rpoints, ptvalues, (X,Y), method='nearest')

## Plot the interpolated data

Code is the same as above, only now we plot interpolated data rather than Theis calculated at every cell.

In [6]:
# control image size
pylab.rcParams['figure.figsize'] = 10,10
# create spatial image of interpolated data
im = pylab.imshow(grid_z0,cmap=pylab.cm.YlOrRd,origin='lower')
# now we contour the data in intervals of 5 from 0 to 200
cset = pylab.contour(grid_z0,np.arange(0,200,5))
# now we display the color bar
pylab.colorbar(im)
# finally we add a title to our image
pylab.title('Theis solution')
pylab.show()

## Finally, let's compare the Theis solution calculated at each point with the interpolated surface from the random points

This is achieved by taking a difference between the two surfaces.

In [7]:
# calculate difference between "real" and "interpolated" surface
diff = values-grid_z0
# control image size
pylab.rcParams['figure.figsize'] = 10,10
# plot difference
# we use the seismic color bar to ensure that white is the middle color
# we control the range over which the color map plots vmin and vmax; assigned such that 0 corresponds with white
# in other words, white indicates the same values
im = pylab.imshow(diff,cmap=pylab.cm.seismic, vmin = -10, vmax = 10,origin='lower')
pylab.colorbar(im)
pylab.show()

## A quick detour: Dictionaries

A dictionary is a collection which is unordered, changeable and indexed. In Python dictionaries are written with curly brackets, and they have keys and values.

Some interactive tutorials: https://www.w3schools.com/python/python_dictionaries.asp

More in-depth tutorials and discussion about Python data types: https://www.datacamp.com/community/tutorials/python-dictionary-tutorial

In [8]:
# Example Dictionary: Big Ten Football Champions
B10Champs = {'2014': ['Ohio State','14-1'], '2015': ['Michigan State','12-2'], '2016': ['Penn State','11-3'], '2017': ['Ohio State','12-2'], '2018': ['Ohio State','13-1']}

In [9]:
print(B10Champs['2016'])

['Penn State', '11-3']


In [10]:
for key, value in B10Champs.items():
    print('The', key, 'Big 10 Football champion was',value[0],'with a record of',value[1])

The 2014 Big 10 Football champion was Ohio State with a record of 14-1
The 2015 Big 10 Football champion was Michigan State with a record of 12-2
The 2016 Big 10 Football champion was Penn State with a record of 11-3
The 2017 Big 10 Football champion was Ohio State with a record of 12-2
The 2018 Big 10 Football champion was Ohio State with a record of 13-1


## Obtaining real-time data from the USGS

Let's extract real-time data from a USGS monitoring well in Lee County, a heavily irrigated area of Illinois

https://waterdata.usgs.gov/il/nwis/uv/?site_no=413743089310102&PARAmeter_cd=72019,72020

To start, visit the following page and create a link to gather the appropriate data:

https://waterservices.usgs.gov/rest/IV-Test-Tool.html

We want the last 500 days of depth-to-water data (72019) for the site 413743089310102.

## Let's first create the URL we are going to extract data from

In [11]:
siteNum = "413743089310102"
period_of_days = "500"
USGSparameter = "72019"

# URL with streamflow/groundwater data from the USGS
url = 'https://waterservices.usgs.gov/nwis/iv/?format=json&sites=' + siteNum + '&period=P'+ period_of_days +'D&parameterCd=' + USGSparameter + '&siteStatus=all'

## Let's open the URL and read the data in

First, we have to request to open the url, using urllib.request.urlopen()

Next, we have to load the json-formatted data, which will be read in as a dictionary

In [12]:
#open the URL
response = urllib.request.urlopen(url)
# read the data in as a dictionary
gw_data = json.loads(response.read())

## Now let's extract the data from the dictionary that we actually need

In [13]:
# extract the list of dictionaries containing time series and value information
gw_data2 = gw_data['value']['timeSeries'][0]['values'][0]['value']
# extract the site name
sitename = gw_data['value']['timeSeries'][0]['sourceInfo']['siteName']

## Next create a list for the time series and values

In [14]:
# Initialize arrays to store time and gage data
gw_gage = []
gw_time = [] # Datatype needs to be datetime here
# Add data from list of dictionaries into an array for time and gage
# note that gage requires conversion to a date time
for x in gw_data2:
    # add gage data, must convert from string to float
    gw_gage.append(float(x['value']))
    # extract date
    date = x['dateTime']
    datestr = date[:-10] # Don't include the last ten digits
    dat_time = datetime.strptime(datestr, '%Y-%m-%dT%H:%M:%S') # Convert string to datetime
    # append date time
    gw_time.append(dat_time)



## Finally let's plot the data

In [15]:
# control size of plot
plt.rcParams['figure.figsize'] = 10,10
# plot time vs gage (depth to water)
plt.plot(gw_time,gw_gage)
# add title with sitename and date
plt.title(sitename + ' \n ' + 'Most recent measurement: ' + str(dat_time), fontweight = 'bold')
plt.xticks(rotation=30)
plt.xlabel('date')
plt.ylabel('USGS Parameter '+USGSparameter)
if USGSparameter == '72019':
    plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Question:

Irrigation had the greatest influence on this monitoring well in which of the last three years?